# Decision Tree 

Hey Aayush!

You should really see the implementation of the Decision Tree below. But **this is not a complete** version of the tree. It works well, like uses the recursion and can calculate the gini, but ***where to stop*** is not clear in my mind, so I left it. 

I gave a couple of days and I feel like I should avoid it. In the book, very simple example is given and also the dataset with only 2 features and even both of them are the numerical. So the book doesn't explain how to deal with multiple types of features and how to make the children. 

I tried here to implement in the way I found from other sources. There were many things to consider:
- Features with numbers and categories
- If categories: check for how many are there ie. Yes, No, Maybe
- How many target variables are there ie. Dog, Horse, Cat

I have successfully counted Gini, filtered Numerical and Categorical, Connected Nodes, Leafs, Traversal... and main Recursion. But when to declare the leaf was not clear. So I left the algorithm as it is.

You will find the code in different cells to get the output of each and then also the class and whole function of them. 

Please review it patiently. I have provided the docstrings.

Thanks, <br>
Aayush ∞ Shah

In [1]:
import pandas as pd, numpy as np
from collections import defaultdict

In [2]:
# Creating a simple dataset with mixed features and 3 cateorical 3 targets

length = [2, 5, 3, 10, 5, 6, 2, 12, 0]
cheese = ["Yes", "No", "No", "Yes", "Maybe", "Maybe", "Yes", "Yes", "No"]
who = ["Dog", "Cat", "Cat", "Cat", "Horse", "Horse", "Cat", "Dog", "Dog"]

df = pd.DataFrame({"x1":length,
              "x2":cheese,
              "who":who})
df

,x1,x2,who
0,2,Yes,Dog
1,5,No,Cat
2,3,No,Cat
3,10,Yes,Cat
4,5,Maybe,Horse
5,6,Maybe,Horse
6,2,Yes,Cat
7,12,Yes,Dog
8,0,No,Dog


In [89]:
def numerical_gini(df, feature, target):
    """
    This function will calculate the numerical gini
    for that target. 
    
    • Only numerical feature has to be passed.
    • One feature at a time
    
    Returns
    -------
    ginis: A dict with the `value` and the gini if the
        split has to happen with that value
        
    filters: A list of indices LEFT and RIGHT if the split
        has to happend with that value.
    """
    mapping = defaultdict(dict)
    filtered = defaultdict(dict)
    
    for xi in df[feature]:
        LEFT = df[df[feature] < xi]
        RIGHT = df[df[feature] >= xi]
        
        LEFT_sum = sum(LEFT.value_counts(normalize=True).apply(lambda x: (x * (1 - x))))
        RIGHT_sum = sum(RIGHT.value_counts(normalize=True).apply(lambda x: (x * (1 - x)))) 
        
        mapping[xi] = LEFT_sum + RIGHT_sum
        filtered[xi] = {"LEFT":LEFT.index.tolist(), "RIGHT":RIGHT.index.tolist()}
    return mapping, filtered

In [48]:
ginis, filters = numerical_gini(df, "x1", "who")

In [90]:
def categorical_gini(df, feature, target):
    """
    This function will calculate the categorical gini
    for that target. 
    
    • Only categorical feature has to be passed.
    • One feature at a time
    
    Returns
    -------
    ginis: A dict with the `value` and the gini if the
        split has to happen with that value
        
    filters: A list of indices LEFT and RIGHT if the split
        has to happend with that value.
    """
    
    mapping = defaultdict(dict)
    filtered = defaultdict(dict)
    
    for xi in df[feature].unique():
        LEFT = df[df[feature] == xi]
        RIGHT = df[df[feature] != xi]
        
        LEFT_sum = sum(LEFT.value_counts(normalize=True).apply(lambda x: (x * (1 - x))))
        RIGHT_sum = sum(RIGHT.value_counts(normalize=True).apply(lambda x: (x * (1 - x)))) 
        
        mapping[xi] = LEFT_sum + RIGHT_sum
        filtered[xi] = {"LEFT":LEFT.index.tolist(), "RIGHT":RIGHT.index.tolist()}
    return mapping, filtered

In [177]:
ginis, filters = numerical_gini(df, "x2", "who")

# 

# 

In [5]:
class Node:
    """
    This class is to make Root node and Middle Node
    This will contain:
    → Uplink (direct)
    → Feature Name (X1, X2 ... )
    → Value (Based on what will be the split decided)
    → Gini (The gini of that split)
    → Dtype (Numerical or Categorical)
    → Filter (The indices for LEFT and RIGHT split)
    → Level (On which level the node is at)

    """
    
    def __init__(self, uplink, feature_name, value, gini, 
                 dtype, filter_, level):
        if uplink: self.name = 'Middle-Node'
        else: self.name = 'Root-Node'
            
        self.uplink = uplink
        self.feature = feature_name
        self.value = value
        self.gini = gini
        self.dtype = "Number" if dtype == True else "Categorical"
        self.data_filter = filter_
        self.level = level
    
    
    def set_child(self, child, side):
        if side == "LEFT":
            self.LEFT_child = child
        elif side == "RIGHT":
            self.RIGHT_child = child
    
    
    def __repr__(self):
        if self.dtype == "Number":
            face = f"{self.feature} < {self.value}"
        else:
            face = f"{self.feature} == {self.value}"
        return f"""
        {self.name.upper()}
        -----------
        Feature: {face}
        Gini: {self.gini}
        Dtype: {self.dtype}
        Level: {self.level}"""

In [91]:
class leaf_node:
    """
    This is the leaf node, very las node in the tree.
    I have made a different class for this unlike a 
    common class as I have made for Root and Middle.
    
    It will take uplink,    category, level.
                  ↑            ↑         ↑
                Root/Middle   Final    Which
                              Value    Level
    """
    def __init__(self, uplink, category, level):
        self.uplink = uplink
        self.category = category
        self.level = level
        self.name = "Leaf-node"
    
    def __repr__(self):
        return f"""
        LEAF NODE
        ---------
        Uplink: {self.uplink.name}
        Uplink-Level: {self.uplink.level}
        Level: {self.level}
        Category: {self.category}"""

In [92]:
def is_numerical(dtype):
    """A reusable function to check whether
    a datatype of that feature belongs to
    be a Numerical or Categorical
    """
    if np.issubdtype(dtype, np.number):
        return True
    return False

In [53]:
ginis

defaultdict(dict,
            {2: 0.875,
             5: 1.5500000000000003,
             3: 1.5,
             10: 1.3571428571428572,
             6: 1.5,
             12: 0.875,
             0: 0.8888888888888888})

In [76]:
def get_lowest_gini_feature(df):
    """
    This is a reusable function and will be called each time
    when we want to decide the best split value across all features
    and all values.
    
    Returns
    -------
    
    feature name, metadata (dict)
    """
    minimum_gini_feature_wise = defaultdict(dict)
    for feature in df.columns[:-1]:    
        if is_numerical(df[feature].dtype):
            dtype = True
            ginis, filters = numerical_gini(df, feature, "who")
        else:
            dtype = False
            ginis, filters = categorical_gini(df, feature, "who")

        try:
            value, gini = sorted(ginis.items(), key=lambda x: x[1])[0]
            minimum_gini_feature_wise[feature] = {"value":value,
                                              "gini":gini,
                                              "filter":filters[value],
                                              "dtype":dtype}

            for_minimum = []
            for feature in minimum_gini_feature_wise:
                tup = (feature, minimum_gini_feature_wise[feature]["gini"])
                for_minimum.append(tup)
            feature_to_take = sorted(for_minimum, key=lambda x: x[1])[0][0]
            return feature_to_take, minimum_gini_feature_wise[feature_to_take]
        except: pass    

In [77]:
to_take = get_lowest_gini_feature(df)
to_take

('x1',
 {'value': 2,
  'gini': 0.875,
  'filter': {'LEFT': [8], 'RIGHT': [0, 1, 2, 3, 4, 5, 6, 7]},
  'dtype': True})

In [78]:
def create_node_with(feature_data, parent, level):
    """This will make a Root Node / Middle Node
    based on the calculated gini. This will get the
    data from `get_lowest_gini()` and will be called
    from the recursive function `create_hierarchy()`.
    """
    
    feature = feature_data[0]
    value = feature_data[1]['value']
    gini = feature_data[1]['gini']
    filter_ = feature_data[1]['filter']
    dtype = feature_data[1]['dtype']
    
    node = Node(parent, feature, value, gini, dtype, filter_, level)
    return node

In [79]:
root = create_node_with(to_take, [], 0)

In [80]:
root.data_filter

{'LEFT': [8], 'RIGHT': [0, 1, 2, 3, 4, 5, 6, 7]}

In [34]:
# level = 1
# def create_hierarchy(parent, df, target):
#     global level
#     level += 1
#     print(level)
#     LEFT_index = parent.data_filter["LEFT"]
#     RIGHT_index = parent.data_filter["RIGHT"]
    
#     if level > 100: return
    
#     if len(LEFT_index) <= 1:
#         leaf = leaf_node(uplink=parent, 
#                          category=df.loc[LEFT_index][target].unique(), 
#                          level=level)
#         parent.set_LEFT_child(leaf)
#     else:
#         dfL = df.loc[LEFT_index]
#         feature_to_take = get_lowest_gini_feature(dfL)
#         node = create_node_with(feature_to_take, parent, level)
#         parent.set_LEFT_child(node)
#         create_hierarchy(node, dfL, target)
        
#     if len(RIGHT_index) <= 1:
#         leaf = leaf_node(uplink=parent,
#                          category=df.loc[RIGHT_index][target].unique(),
#                          level=level)
#         parent.set_RIGHT_child(leaf)
#     else:
#         dfR = df.loc[RIGHT_index]
#         feature_to_take = get_lowest_gini_feature(dfR)
#         node = create_node_with(feature_to_take, parent, level)
#         parent.set_RIGHT_child(node)
#         create_hierarchy(node, dfR, target)

The error can be there if the repeated numerical values is found in the feature. Let's see what. Not implemented till now.

In [81]:
def create_hierarchy(parent, df, target):
    """
    This is the major and the hero function in all cases.
    This is the recursive function which eventully uses
    all functions and classes discussed before.
    
    Here the connection and split decision is successful but
    the stopping condition is somewhere not perfect and sometimes
    target class comes null.
    
    The code breakes and gives error. We need to reconfigure things.
    For that, need to understand how to stop.
    """
    
    global level
    global unique_category_per_feature
    level += 1
    print(level)
    
    def check_and_link(index, side):
        dfLR = df.loc[index]
        categorical_gini = dfLR[target].value_counts()
        
        if len(categorical_gini) == 1: # Only one feature in the split
            leaf = leaf_node(uplink=parent,
                             category=categorical_gini.index[0],
                             level=level)
            parent.set_child(leaf, side)
        else: # Means there are more than 1 features
            if not are_there_features(dfLR): # Means no features left, just now make a leaf
                leaf = leaf_node(uplink=parent,
                             category=categorical_gini.index[0],
                             level=level)
                parent.set_child(leaf, side)
            else: # Means there are features left
                feature_to_take = get_lowest_gini_feature(dfLR)
                if feature_to_take[1]["gini"] <= 0.1: # If the gini is low then, directly make a leaf
                    leaf = leaf_node(uplink=parent,
                             category=categorical_gini.index[0],
                             level=level)
                    parent.set_child(leaf, side)
                else:
                    feature_to_take = get_lowest_gini_feature(dfLR)
                    unique_category_per_feature[feature_to_take[0]].remove(feature_to_take[1]['value'])
                    node = create_node_with(feature_to_take, parent, level)
                    parent.set_child(node, side)
                    create_hierarchy(node, dfLR, target)
            
            
    LEFT_index = parent.data_filter["LEFT"]
    RIGHT_index = parent.data_filter["RIGHT"]
    
    if level > 100: return
    check_and_link(LEFT_index, "LEFT")
    check_and_link(RIGHT_index, "RIGHT")

Now, some experiment code will start. So, you might not get the docstrings!

In [82]:
categories = []
for feature in df.columns[:-1]:
    if not is_numerical(df[feature].dtype):
        categories.append(feature)

In [83]:
categories

['x2']

In [84]:
categories = []
for feature in df.columns[:-1]:
    if not is_numerical(df[feature].dtype):
        categories.append(feature)

def are_there_features(df):
    for feature in df.columns[:-1]:
        if len(unique_category_per_feature[feature]) > 1:
            return True
    return False

In [85]:
# This will make the initial list of all feature (cat and num)
# features and the unique categories, which will be
# removed one by one in the `create_hierarcy()`

unique_category_per_feature = {}
for feature in df.columns[:-1]:
    unique_category_per_feature[feature] = df[feature].unique().tolist()
    
root = create_node_with(to_take, [], 0)
level = 1

create_hierarchy(root, df, "who")

2
3
4
5
6


TypeError: 'NoneType' object is not subscriptable

In [86]:
level = 0
def traverse(node):
    global level
    level += 1
    print(level)
    if node.name == "Leaf-node":
        print("\t" * (level), node)
        return
    else:
        print("\t", "\nLEFT ", node)
        traverse(node.LEFT_child)
        
        print("\t", "\nRIGHT ", node)
        traverse(node.RIGHT_child)

In [38]:
traverse(root)

1
	 
LEFT  
        ROOT-NODE
        -----------
        Feature: x1 < 2
        Gini: 0.875
        Dtype: Number
        Level: 0
2
		 
        LEAF NODE
        ---------
        Uplink: Root-Node
        Uplink-Level: 0
        Level: 2
        Category: ['Dog']
	 
RIGHT  
        ROOT-NODE
        -----------
        Feature: x1 < 2
        Gini: 0.875
        Dtype: Number
        Level: 0
3
	 
LEFT  
        MIDDLE-NODE
        -----------
        Feature: x1 < 12
        Gini: 0.8571428571428572
        Dtype: Number
        Level: 2
4
	 
LEFT  
        MIDDLE-NODE
        -----------
        Feature: x1 < 10
        Gini: 0.8333333333333333
        Dtype: Number
        Level: 3
5
	 
LEFT  
        MIDDLE-NODE
        -----------
        Feature: x1 < 6
        Gini: 0.8000000000000002
        Dtype: Number
        Level: 4
6
	 
LEFT  
        MIDDLE-NODE
        -----------
        Feature: x2 == Maybe
        Gini: 0.75
        Dtype: Categorical
        Level: 5
7
							 

AttributeError: 'Node' object has no attribute 'LEFT_child'